In [1]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lightgbm.sklearn import LGBMClassifier
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score,f1_score
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

In [2]:
#csv导入数据,转为dataframe
train=pd.read_excel("C:\\Users\\dell\\Desktop\\ML\\train.xlsx" )
test=pd.read_excel("C:\\Users\\dell\\Desktop\\ML\\test.xlsx")

In [3]:
#手动划分训练集和验证集
X_train=train.drop('Shell', axis=1)
X_test=test.drop('Shell',axis=1)
y_train=train.Shell
y_test=test.Shell



In [4]:
#欠采样
rus = RandomUnderSampler(random_state=0, replacement=True)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [5]:
# 生成lgb的Dataset数据格式
train_data = lgb.Dataset(X_train, label=y_train)      #训练集
validation_data = lgb.Dataset(X_test, label=y_test)  #验证集

In [47]:
# 设置超参数
params = {
    'boosting_tpye':'gbdt',   #默认，还有rf
     'objective': 'binary',  # 目标函数
    'metric':{'binary_logloss','auc'},           #metric策略采用的是，越低越好
    "num_boost_round": 200,        #n_estimators/num_iterations/num
    'learning_rate': 0.02, 
    'lambda_l1': 0.01,
    'lambda_l2': 0.01,
    'max_depth': 10,
    'num_leaves':40,           #要小于2 ^max_depth
    'bagging_fraction':0.8,    #subsample，样本采样比例，调小可以防止过拟合，加快运算速度
    'bagging_freq':5,          #colsample_bytree
   # 'num_class': 1,
    'min_child_weight':0.01,    #叶子节点的最小权重和，调大可以防止过拟合
    'min_child_samples':18      #叶子节点中最小的数据量，调大可以防止过拟合
}

# 模型训练

gbm = lgb.train(params, train_data, valid_sets=validation_data)  #n轮后没提升就stop

# 对验证集进行预测
y_pred = gbm.predict(X_test)

print(y_pred)

[LightGBM] [Warning] Unknown parameter: boosting_tpye
[LightGBM] [Warning] Unknown parameter: boosting_tpye
[LightGBM] [Warning] Unknown parameter: boosting_tpye
[LightGBM] [Warning] Unknown parameter: boosting_tpye
[LightGBM] [Warning] Unknown parameter: boosting_tpye
[LightGBM] [Info] Number of positive: 101, number of negative: 100
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 166
[LightGBM] [Info] Number of data points in the train set: 201, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502488 -> initscore=0.009950
[LightGBM] [Info] Start training from score 0.009950
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.744	valid_0's binary_logloss: 0.687689
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.7464	valid_0

f:\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


#保存features

In [45]:
#设置阈值
threshold =0.5
y_pred2 = []
for i in range(len(y_pred)):
    if y_pred[i]>threshold:
        s=1
    else:
        s=0
    y_pred2.append(s)

y_pred2



[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [43]:
# 模型评估(五种指标输出)
print("accuracy为{}".format(accuracy_score(y_test, y_pred2)))
print("auc为{}".format(roc_auc_score(y_test, y_pred2)))
print("F1得分为{}".format(f1_score(y_test, y_pred2)))
print('Precision为{}'.format(precision_score(y_test, y_pred2)))
print("recall {}".format(recall_score(y_test, y_pred2)))



accuracy为0.84
auc为0.84
F1得分为0.8399999999999999
Precision为0.84
recall 0.84


In [9]:
#________Grid SearchCV___________

lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [6,8,10,12,14,16,18,20],
              "learning_rate" : [0.005,0.01,0.02],
              "num_leaves": [40,64,128,256,1024,2048,4096],            
              "n_estimators": [200],
              'lambda_l1': [0.001, 0.02],
              'lambda_l2': [0, 1],
              "subsample":[0.8,0.9,1],
              'subsample_freq':[2,3,4,5],
              "min_child_weight":[0.001,0.002],
              "min_child_samples":[18,19,20,21,22]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, 
                           scoring="recall", verbose=1)  #cv:交叉验证参数
grid_search.fit(train,y_train)

lgbm_best = grid_search.best_params_
print('lgbm Regressor Best Parmas',grid_search.best_params_)
print('lgbm Regressor Best Score',grid_search.best_score_)


Fitting 3 folds for each of 192 candidates, totalling 576 fits
[LightGBM] [Warning] lambda_l1 is set=0.001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Warning] lambda_l1 is set=0.001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
[LightGBM] [Info] Number of positive: 101, number of negative: 100
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 168
[LightGBM] [Info] Number of data points in the train set: 201, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502488 -> initscore=0.009950
[LightGBM] [Info] Start training from score 0.009950
[LightGBM] [Warning] No further spli

f:\Anaconda3\lib\site-packages\lightgbm\sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
